# "Island" catchment areas

Ordinarily the catchment areas of each stroke unit don't match up nicely to other defined regions, for example the Integrated Care Boards or Ambulance Service boundaries.

Sometimes though it's preferable to only consider patients within a given region, even if patients outside the border would still ordinarily travel to a stroke unit within that region.

This is like pretending that everything outside the region doesn't exist. Outside the region, there are no LSOA with patients who will want to enter the region for treatment. Outside the region, there are no other stroke units that patients within the region might prefer to travel to. When the regions are split off like this, it is as though each one is its own island with nothing else surrounding it.

To create this effect with the catchment code, it is possible to restrict:
+ which stroke units are considered for each LSOA
+ which LSOA are considered

In this example we will restrict everything to the ISDN named "West Midlands".


## Notebook setup

In [1]:
import stroke_maps.load_data
import stroke_maps.catchment

import pandas as pd

## Load data

__LSOA travel time__

This dataframe contains one row for each LSOA in England and Wales. There is one column for each stroke unit in England and Wales. The value in each cell is the time from that row's LSOA to that column's stroke unit.

In [2]:
df_travel_lsoa = stroke_maps.load_data.travel_time_matrix_lsoa()

# Show the first five rows and columns:
df_travel_lsoa.iloc[:5, :5]

,B152TH,B714HJ,B95SS,BA13NG,BA214AT
LSOA,,,,,
Adur 001A,173.3,179.8,171.2,161.5,152.9
Adur 001B,173.3,179.8,172.3,161.5,152.9
Adur 001C,173.3,180.9,172.3,150.8,151.9
Adur 001D,173.3,180.9,172.3,161.5,152.9
Adur 001E,174.4,180.9,173.3,150.8,151.9


__Stroke unit services__

This dataframe contains information on which stroke units provide which services (IVT and MT). The imported dataframe, df_units, contains only units that appear in the travel time matrix. This includes units that are not acute stroke units (i.e. provide neither thrombolysis nor thrombectomy).

In [4]:
df_units = stroke_maps.load_data.stroke_unit_region_lookup()

df_units.head(3).T

postcode,SY231ER,CB20QQ,L97AL
stroke_team,Bronglais Hospital (Aberystwyth),"Addenbrooke's Hospital, Cambridge","University Hospital Aintree, Liverpool"
short_code,AB,AD,AI
ssnap_name,Bronglais Hospital,Addenbrooke's Hospital,University Hospital Aintree
use_ivt,1,1,1
use_mt,0,1,1
use_msu,0,1,1
transfer_unit_postcode,nearest,nearest,nearest
lsoa,Ceredigion 002A,Cambridge 013D,Liverpool 005A
lsoa_code,W01000512,E01017995,E01006654
region,Hywel Dda University Health Board,NHS Cambridgeshire and Peterborough ICB - 06H,NHS Cheshire and Merseyside ICB - 99A


Make a copy of the travel time matrix to edit:

In [5]:
df_travel_lsoa = stroke_maps.load_data.travel_time_matrix_lsoa()

df_travel_lsoa_here = df_travel_lsoa.copy()

Limit the stroke units:

In [6]:
mask_units = (
    (df_units['isdn'] == 'West Midlands') &
    (df_units['use_ivt'] == 1)
)

df_units_here = df_units[mask_units]

In [7]:
df_travel_lsoa_here = df_travel_lsoa_here[df_units_here.index]

Load the LSOA-region lookup data:

In [8]:
df_lsoa_regions = stroke_maps.load_data.lsoa_region_lookup()
df_regions = stroke_maps.load_data.region_lookup()

# Merge the ISDN names into the LSOA-region lookup:
df_lsoa_regions = pd.merge(
    df_lsoa_regions.reset_index(),
    df_regions.reset_index()[['region_code', 'isdn']],
    on='region_code', how='left'
)

Pick out the LSOAs in the chosen region:

In [9]:
mask_lsoa = df_lsoa_regions['isdn'] == 'West Midlands'
df_lsoa_regions = df_lsoa_regions[mask_lsoa].copy()

Limit the travel time matrix to only these LSOA:

In [10]:
mask_travel = df_travel_lsoa_here.index.isin(df_lsoa_regions['lsoa'])
df_travel_lsoa_here = df_travel_lsoa_here[mask_travel]

Resulting travel matrix:

In [11]:
df_travel_lsoa_here

,CV22DX,HR12ER,B152TH,WR51DD
LSOA,,,,
Birmingham 001A,36.9,84.2,31.6,47.7
Birmingham 001B,36.9,85.3,33.7,49.8
Birmingham 001C,38.0,84.2,32.6,48.8
Birmingham 001D,36.9,85.3,33.7,49.8
Birmingham 002A,35.9,85.3,33.7,49.8
...,...,...,...,...
Wyre Forest 013D,64.9,57.4,38.0,26.2
Wyre Forest 014A,64.9,52.0,38.0,26.2
Wyre Forest 014B,65.9,52.0,39.1,27.3


Pull out the nearest stroke units:

In [12]:
df_catchment = stroke_maps.catchment.find_nearest_unit(df_travel_lsoa_here)

df_catchment

,unit_travel_time,unit_postcode
LSOA,,
Birmingham 001A,31.6,B152TH
Birmingham 001B,33.7,B152TH
Birmingham 001C,32.6,B152TH
Birmingham 001D,33.7,B152TH
Birmingham 002A,33.7,B152TH
...,...,...
Wyre Forest 013D,26.2,WR51DD
Wyre Forest 014A,26.2,WR51DD
Wyre Forest 014B,27.3,WR51DD


This dataframe contains only stroke units and LSOA within the boundary of the selected region.